# Installs

In [1]:
#!pip install tensorflow_hub


# Imports


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import random 
from tensorflow import keras
from tensorflow.keras import layers

# Definición de emociones a usar

In [3]:
Classes = ["0","1","2","3","4","5"] #Nombre para las clases
Archives = ["angry","disgust","fear","happy","sad","surprise"]
#Las clases serán resectivamente: enojo=0, disgusto=1, miedo=2, felicidad=3, tristeza=4, sorpresa=5

Datadirectory = "train/" #Para saber de donde se van a traer los datos

# Obtención del set de datos

In [4]:
img_size = 224
training_Data = []

#Se accede a cada uno de las carpetas para entrenar cada clase
for category in Classes:
    path = os.path.join(Datadirectory, category)
    class_num = Classes.index(category)
    j = 0
    for img in os.listdir(path):
        if j < 400:
            j+=1
            img_array = cv2.imread(os.path.join(path,img))
            new_array = cv2.resize(img_array, (img_size, img_size))
            training_Data.append([new_array,class_num])

#Desordena para que no haya problemas
random.shuffle(training_Data)
X = []
Y = []

#Agrega en un solo

for features,labels in training_Data:
  X.append(features)
  Y.append(labels)
Y = np.array(Y)
X = np.array(X).reshape(-1,img_size,img_size,3)

#Se normalizan los datos, cuyo valor máximo es 255 (escala de grises)
X = X / 255.0;

# Se genera el modelo

In [5]:
#Se carga un modelo diseño
model = tf.keras.applications.MobileNetV2()

base_input = model.layers[0].input
base_output = model.layers[-2].output
final_output = layers.Dense(128)(base_output)
final_ouput = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_ouput = layers.Activation('relu')(final_output)
final_output = layers.Dense(6,activation='softmax')(final_output)

modelo_caras = keras.Model(inputs = base_input, outputs = final_output)

modelo_caras.compile(
  optimizer="adam",
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

#si se quiere visualizar el modelo final
#modelo_caras.summary()

# Se entrena el modelo

In [6]:
hist = modelo_caras.fit(X,Y,epochs=20)

Epoch 1/20
75/75 [==============================] - 55s 704ms/step - loss: 1.6875 - accuracy: 0.3746
Epoch 2/20
75/75 [==============================] - 53s 703ms/step - loss: 1.1584 - accuracy: 0.5567
Epoch 3/20
75/75 [==============================] - 53s 703ms/step - loss: 0.9841 - accuracy: 0.6288
Epoch 4/20
75/75 [==============================] - 53s 703ms/step - loss: 0.8006 - accuracy: 0.6908
Epoch 5/20
75/75 [==============================] - 53s 704ms/step - loss: 0.7033 - accuracy: 0.7317
Epoch 6/20
75/75 [==============================] - 53s 702ms/step - loss: 0.6128 - accuracy: 0.7825
Epoch 7/20
75/75 [==============================] - 53s 700ms/step - loss: 0.5258 - accuracy: 0.8050
Epoch 8/20
75/75 [==============================] - 53s 703ms/step - loss: 0.4149 - accuracy: 0.8529
Epoch 9/20
75/75 [==============================] - 53s 705ms/step - loss: 0.4225 - accuracy: 0.8558
Epoch 10/20
75/75 [==============================] - 53s 709ms/step - loss: 0.3037 - accura

# Prueba

In [7]:
P=cv2.imread("train/0/Training_3908.jpg")
P=cv2.resize(P,(224,224))
P = np.expand_dims(P,axis=0)
P = P/255.0

prediccion=modelo_caras.predict(P) #Mayor valor es nuestra predicción
print(prediccion)

1/1 [==============================] - 0s 321ms/step
[[1.0000000e+00 2.6598330e-16 2.2132438e-17 3.9089520e-16 2.2175371e-17
  4.1378434e-09]]


# Modelo TFlite

In [8]:
!mkdir "tflite_models"

#Path y nombre del modelo tflite

TFLITE_MODEL = "tflite_models/modelo_caras.tflite"

run_model = tf.function(lambda x : modelo_caras(x))
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

#Conversión a tflite

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

9548408

# Carga modelo TFlite

In [10]:
#Path y nombre del modelo tflite
TFLITE_MODEL = "tflite_models/modelo_caras.tflite"

tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)

tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

# Prueba TFLite

In [11]:
#Se requiere que el elemento a predecir sea tipo float32
img = np.float32(P)

tflite_interpreter.set_tensor(input_details[0]['index'], img)
tflite_interpreter.invoke()
tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
print(tflite_model_predictions)

[[1.0000000e+00 2.6598431e-16 2.2132269e-17 3.9089366e-16 2.2175200e-17
  4.1378354e-09]]
